<a href="https://colab.research.google.com/github/Tejshah740/AI-ML_tutor_chatbot/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch transformers

In [ ]:
from transformers import BertTokenizer
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
text='hey there'
tokens=tokenizer.tokenize(text)
print(tokens)

['hey', 'there']


In [ ]:
ids=tokenizer.convert_tokens_to_ids(tokens)
print(ids)

[4931, 2045]


In [ ]:
encoding=tokenizer(text,
                   return_tensors="pt")
print(encoding)

{'input_ids': tensor([[ 101, 4931, 2045,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}


In [ ]:
texts=[
    " i love this movie",
    " i dont love this movie",
    " the movie was awesome but the plot was slow",
    " the movie was bad but the songs were good"
]
labels=[1,0,1,0]


In [ ]:
encodings=tokenizer(texts,
                   padding=True,
                   truncation=True,
                   return_tensors="pt")
print(encodings)

{'input_ids': tensor([[  101,  1045,  2293,  2023,  3185,   102,     0,     0,     0,     0,
             0],
        [  101,  1045,  2123,  2102,  2293,  2023,  3185,   102,     0,     0,
             0],
        [  101,  1996,  3185,  2001, 12476,  2021,  1996,  5436,  2001,  4030,
           102],
        [  101,  1996,  3185,  2001,  2919,  2021,  1996,  2774,  2020,  2204,
           102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
import torch
labels=torch.tensor(labels)
print(labels)

tensor([1, 0, 1, 0])


In [ ]:
from transformers import BertForSequenceClassification
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
outputs=model(input_ids=encodings["input_ids"],
              attention_mask=encodings["attention_mask"],
              labels=labels)

In [ ]:
print(outputs.loss)

tensor(0.5732, grad_fn=<NllLossBackward0>)


In [ ]:
model.train()
from torch.optim import AdamW
optimiser=AdamW(model.parameters(),lr=5e-5)


In [ ]:
epochs=10
for epoch in range(epochs):
  optimiser.zero_grad()
  outputs=model(input_ids=encodings["input_ids"],
              attention_mask=encodings["attention_mask"],
              labels=labels)
  loss=outputs.loss
  loss.backward()
  optimiser.step()

  print(loss)

tensor(0.1500, grad_fn=<NllLossBackward0>)
tensor(0.1193, grad_fn=<NllLossBackward0>)
tensor(0.1049, grad_fn=<NllLossBackward0>)
tensor(0.0963, grad_fn=<NllLossBackward0>)
tensor(0.0884, grad_fn=<NllLossBackward0>)
tensor(0.0709, grad_fn=<NllLossBackward0>)
tensor(0.0575, grad_fn=<NllLossBackward0>)
tensor(0.0389, grad_fn=<NllLossBackward0>)
tensor(0.0306, grad_fn=<NllLossBackward0>)
tensor(0.0336, grad_fn=<NllLossBackward0>)


In [ ]:
model.eval()
new_text="i like the movie but songs were not good"
encodings=tokenizer(new_text,return_tensors='pt',truncation=True,padding=True)


In [ ]:

with torch.no_grad():
  outputs=model(input_ids=encodings["input_ids"],
                attention_mask=encodings["attention_mask"])

In [ ]:
logits=outputs.logits
prediction=torch.argmax(logits,dim=1)
if prediction==1:
  print("positive")
else:
  print("negative")

negative
